In [ ]:
import os
import numpy as np
import pandas as pd
import torch 
import cv2
import torch
import torch.nn as nn
from PIL import Image
from torch.autograd import Variable
from torchvision import models, transforms
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm 
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text

# Video Feature extraction

In [ ]:
# --- CẤU HÌNH ---
input_folder = r'D:\learn\giaotrinh\ky7\CV\final\downloads\scripts'
output_feature_file = r'D:\learn\giaotrinh\ky7\CV\final\MMRec\data\text_features_minilm_384d.npy'
output_id_file = r'D:\learn\giaotrinh\ky7\CV\final\MMRec\data\text_ids.npy' 

# 1. Tải Model Transformer nhẹ
print("Loading Sentence-Transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Quét file và Sắp xếp (QUAN TRỌNG NHẤT)
# Lấy tất cả file .txt
files = [f for f in os.listdir(input_folder) if f.endswith('.txt')]

# Sắp xếp danh sách file theo tên (alphabet/numerical)
# Điều này đảm bảo: file '000123.txt' sẽ được xử lý trước '000124.txt'
# Nếu bên Video bạn cũng sort như thế này, thì 2 bên sẽ khớp nhau 100%
files.sort() 

print(f"Found {len(files)} text files. Processing in sorted order...")

# 3. Đọc dữ liệu
all_texts = []
all_ids = []

for filename in tqdm(files):
    file_path = os.path.join(input_folder, filename)
    
    # Lấy ID từ tên file (bỏ đuôi .txt)
    # Ví dụ: '102394.txt' -> '102394'
    file_id = filename.replace('.txt', '')
    all_ids.append(file_id)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        # Nếu file rỗng thì để chuỗi rỗng (model vẫn xử lý được)
        all_texts.append(content)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        all_texts.append("") # Append rỗng để giữ đúng index

# 4. Trích xuất đặc trưng (Batch Processing)
print("Encoding texts...")
# batch_size=64 hoặc 128 tùy vào VRAM của GPU trên Kaggle
features_array = model.encode(all_texts, batch_size=128, convert_to_numpy=True, show_progress_bar=True)

# 5. Lưu kết quả
# Lưu ma trận đặc trưng
np.save(output_feature_file, features_array)

# Lưu danh sách ID (để sau này map với item)
np.save(output_id_file, np.array(all_ids))

print("--- HOÀN TẤT ---")
print(f"Saved features shape: {features_array.shape}") # (Số lượng file, 384)
print(f"Saved IDs list shape: {len(all_ids)}")
print(f"Files saved: {output_feature_file} and {output_id_file}")

# Kiểm tra thử 3 ID đầu tiên
print("First 3 IDs processed:", all_ids[:3])

In [ ]:
# --- CẤU HÌNH ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
video_dir1 = r"D:\learn\giaotrinh\ky7\CV\final\downloads\videos" 
output_feature_file = r"D:\learn\giaotrinh\ky7\CV\final\MMRec\data\video_features_uniform_concat.npy"
output_id_file = r"D:\learn\giaotrinh\ky7\CV\final\MMRec\data\video_ids.npy"

# --- HÀM HỖ TRỢ ---
def _load_video(video_dir, video_id):
    """Load and uniformly sample 4 frames from video."""
    num_frames = 4
    video_name = str(video_id) + ".mp4"
    path = os.path.join(video_dir, video_name)
    
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise Exception(f"Cannot open video {video_name}")
        
    NumFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Chiến lược lấy mẫu
    if NumFrames < num_frames:
        # Nếu video quá ngắn hoặc lỗi đọc frame, trả về 0
        if NumFrames <= 0:
             return torch.zeros(num_frames, 3, 224, 224)
        indices = np.linspace(0, NumFrames - 1, num_frames, dtype=int)
    else:
        indices = np.linspace(0, NumFrames - 1, num_frames, dtype=int)

    sampled_frms = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        rval, frame = cap.read()
        if rval:
            # Convert BGR (OpenCV) to RGB (PIL/PyTorch)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            img = img.resize((224, 224))
            img = transforms.ToTensor()(img)
            img = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])(img)
            sampled_frms.append(img)
        else:
            sampled_frms.append(torch.zeros(3, 224, 224))
    
    cap.release()
    
    # Stack lại: [4, 3, 224, 224]
    return torch.stack(sampled_frms, dim=0)

class GridFeatBackbone(nn.Module):
    def __init__(self):
        super(GridFeatBackbone, self).__init__()
        # Load ViT
        self.net = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        self.net.heads = nn.Identity()

    def forward(self, x):
        # x shape: [batch_size * num_frames, 3, 224, 224]
        return self.net(x)

# --- QUY TRÌNH CHÍNH ---

# 1. Lấy danh sách file và SẮP XẾP CHUẨN (String Sort)
# Lưu ý: Dùng sort() thường để khớp với code Text bên trên (001.txt, 002.txt...)
files = [f for f in os.listdir(video_dir1) if f.endswith('.mp4')]
files.sort() 

print(f"Found {len(files)} videos. Starting extraction on {device}...")

# 2. Khởi tạo model
model = GridFeatBackbone().to(device)
model.eval() # Quan trọng: Chuyển sang chế độ đánh giá (tắt dropout)

all_features = []
processed_ids = []

# 3. Vòng lặp với tqdm gọn gàng
# desc: Tiêu đề thanh loading
for f in tqdm(files, desc="Extracting Visual Features"):
    video_id = f.replace('.mp4', '') # Lấy ID, ví dụ "123456"
    
    try:
        # Load frames: [4, 3, 224, 224]
        frms = _load_video(video_dir1, video_id)
        frms = frms.to(device)
        
        with torch.no_grad():
            # ViT nhận batch. Ở đây 4 frame coi như batch=4
            out = model(frms) # [4, 768]
            
            # Flatten: [4, 768] -> [1, 3072]
            # view(1, -1) nhanh hơn và gọn hơn cách dùng torch.cat loop cũ
            video_feature = out.reshape(1, -1) 
            
        # Move về CPU để lưu RAM
        all_features.append(video_feature.cpu().numpy())
        processed_ids.append(video_id)
        
    except Exception as e:
        # Dùng tqdm.write để in lỗi không bị vỡ thanh loading
        tqdm.write(f"Error processing {video_id}: {e}")
        # Nếu lỗi, thêm vector 0 để giữ đúng index (quan trọng để khớp với Text)
        all_features.append(np.zeros((1, 768 * 4), dtype=np.float32))
        processed_ids.append(video_id)

# 4. Lưu file
if len(all_features) > 0:
    all_features = np.concatenate(all_features, axis=0)
    np.save(output_feature_file, all_features)
    np.save(output_id_file, np.array(processed_ids)) # Lưu luôn ID cho chắc
    
    print("\nDone!")
    print(f"Features shape: {all_features.shape}") # (N, 3072)
    print(f"Saved to: {output_feature_file}")
else:
    print("No features extracted.")

# User feature extraction

In [9]:
import pandas as pd
interact_df = pd.read_csv(r'D:\learn\giaotrinh\ky7\CV\final\data\raw_items\interaction_final.csv')
len(interact_df)

7491

In [10]:
interact_df.columns

Index(['user_id', 'pid', 'author_id', 'category_id', 'category_level',
       'parent_id', 'root_id', 'exposed_time', 'author_fans_count',
       'watch_time', 'duration', 'cvm_like', 'click', 'comment', 'follow',
       'collect', 'forward', 'hate', 'tag_name', 'title', 'p_hour', 'p_date',
       'gender', 'age', 'mod_price', 'fre_city', 'fre_community_type',
       'fre_city_level'],
      dtype='object')

In [11]:
columns_to_keep = ['user_id', 'pid', 'exposed_time', 'p_date', 'p_hour', 'watch_time', 'cvm_like', 'comment', 'follow', 
                   'collect', 'forward', 'hate', 'gender', 'age', 'mod_price', 'fre_city', 'fre_community_type', 'fre_city_level', 'duration']
interact_df = interact_df[columns_to_keep]


In [12]:
interact_df.duplicated().sum()

np.int64(6792)

In [14]:
interact_df = interact_df.drop_duplicates()
len(interact_df)

699

In [19]:
interact_df.to_csv('interaction_final_cleaned.csv', index=False)

In [15]:
interact_df.user_id.nunique()

170

In [16]:
interact_df.groupby('user_id').size().mean()

np.float64(4.1117647058823525)

In [17]:
distinct_per_user = interact_df.groupby('user_id')['pid'].nunique()
mean_distinct = distinct_per_user.mean()
print(mean_distinct)

4.041176470588235


In [18]:
interact_df.head()

,user_id,pid,exposed_time,p_date,p_hour,watch_time,cvm_like,comment,follow,collect,forward,hate,gender,age,mod_price,fre_city,fre_community_type,fre_city_level,duration
0,366,66812,1663337736,20220916,22,29,False,False,False,False,False,False,F,69,899,淮安,城区,三线城市,36.816
15,366,57413,1663337682,20220916,22,7,False,False,False,False,False,False,F,69,899,淮安,城区,三线城市,268.626
27,366,59348,1663337736,20220916,22,21,False,False,False,False,False,False,F,69,899,淮安,城区,三线城市,255.346
39,2494,52190,1663336889,20220916,22,5,False,False,False,False,False,False,M,55,2699,巴音郭楞蒙古自治州,unknown,五线城市,215.433
49,2494,68485,1663337405,20220916,22,17,False,False,False,False,False,False,M,55,2699,巴音郭楞蒙古自治州,unknown,五线城市,183.833


In [79]:
interact_df.columns

Index(['user_id', 'pid', 'exposed_time', 'p_date', 'p_hour', 'watch_time',
       'cvm_like', 'comment', 'follow', 'collect', 'forward', 'hate', 'gender',
       'age', 'mod_price', 'fre_city', 'fre_community_type', 'fre_city_level',
       'duration'],
      dtype='object')

In [80]:
interact_df.describe()

,user_id,pid,exposed_time,p_date,p_hour,watch_time,age,mod_price,duration
count,7491.000000,7491.000000,7.491000e+03,7491.0,7491.000000,7491.000000,7491.000000,7491.000000,7491.000000
mean,4748.159258,61420.417701,1.663315e+09,20220916.0,15.688159,70.863970,45.594046,2188.335469,172.217237
std,2899.808565,30601.328875,2.020604e+04,0.0,5.619387,85.214609,17.043753,2026.743777,130.208301
min,2.000000,46.000000,1.663264e+09,20220916.0,2.000000,0.000000,20.000000,400.000000,6.000000
25%,2255.000000,34115.000000,1.663300e+09,20220916.0,11.000000,9.000000,32.000000,1099.000000,72.680000
50%,4716.000000,66191.000000,1.663320e+09,20220916.0,17.000000,35.000000,45.000000,1599.000000,149.004000
75%,7192.000000,88982.500000,1.663333e+09,20220916.0,21.000000,107.000000,59.000000,2499.000000,259.492000
max,9939.000000,100959.000000,1.663343e+09,20220916.0,23.000000,612.000000,79.000000,17799.000000,821.800000


In [81]:
# Danh sách các cột
binary_cols = ['cvm_like', 'click', 'comment', 'follow', 'collect', 'forward', 'hate']

# Lọc chỉ lấy các cột có trong df
existing_cols = [c for c in binary_cols if c in interact_df.columns]

# Dùng apply để tính value_counts cho nhiều cột cùng lúc
summary_table = interact_df[existing_cols].apply(pd.Series.value_counts).T

print(summary_table.fillna(0).astype(int))

          False  True 
cvm_like   7275    216
comment    7479     12
follow     7402     89
collect    7463     28
forward    7479     12
hate       7491      0


In [82]:
positive_df = interact_df[
    (interact_df['watch_time'] > 30) | 
    (interact_df['click'] == 1) |
    (interact_df['cvm_like'] == 1)
].copy()

KeyError: 'click'

In [ ]:
class UserTower(tf.keras.Model):
    def __init__(self, user_ids, city_vocab):
        super().__init__()
        
        # 1. Embedding cho User ID
        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(user_ids) + 1, 32)
        ])

        # 2. Embedding cho City (Categorical Feature)
        self.city_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=city_vocab, mask_token=None),
            tf.keras.layers.Embedding(len(city_vocab) + 1, 16)
        ])

        # 3. Xử lý Age (Numerical Feature)
        # Chuẩn hóa age về khoảng nhỏ (chia cho 100 hoặc dùng Normalization layer)
        self.normalized_age = tf.keras.layers.Normalization(axis=None)

        # 4. Dense Layer cuối cùng để trộn tất cả
        self.dense = tf.keras.layers.Dense(32) # Output dimension = 32

    def call(self, inputs):
        # Lấy embedding các phần
        u_vec = self.user_embedding(inputs["user_id"])
        c_vec = self.city_embedding(inputs["fre_city"])
        
        # Age cần reshape để nối được
        age_val = tf.reshape(self.normalized_age(inputs["age"]), (-1, 1))

        # Nối tất cả lại: [UserID_32 + City_16 + Age_1]
        concatenated = tf.concat([u_vec, c_vec, age_val], axis=1)
        
        # Qua lớp Dense để ra vector cuối cùng (32 chiều)
        return self.dense(concatenated)

# Lấy vocab để init model
unique_user_ids = np.unique(positive_df['user_id'].astype(str).values)
unique_cities = np.unique(positive_df['fre_city'].astype(str).values)

user_model = UserTower(unique_user_ids, unique_cities)
# Adapt layer normalization cho age
user_model.normalized_age.adapt(positive_df['age'].fillna(0).values)

In [ ]:
class ItemTower(tf.keras.Model):
    def __init__(self, item_ids):
        super().__init__()

        # 1. Embedding cho PID (ID Video)
        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=item_ids, mask_token=None),
            tf.keras.layers.Embedding(len(item_ids) + 1, 32)
        ])

        # 2. Xử lý Vector Content (Input size 128 -> Project xuống 32)
        self.content_projection = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(32)
        ])

        # 3. Dense trộn ID và Content
        self.dense = tf.keras.layers.Dense(32) # Output dimension = 32 (Phải khớp User Tower)

    def call(self, inputs):
        # inputs["pid"] và inputs["item_vector"]
        
        id_vec = self.item_embedding(inputs["pid"])
        content_vec = self.content_projection(inputs["item_vector"])

        # Cộng hoặc Nối. Ở đây mình dùng Nối (Concat) rồi Dense
        concatenated = tf.concat([id_vec, content_vec], axis=1)
        
        return self.dense(concatenated)

unique_item_ids = np.unique(df_video['pid'].astype(str).values)
item_model = ItemTower(unique_item_ids)

In [ ]:
class YouTubeRetrievalModel(tfrs.models.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        
        # Task Retrieval: Tự động tính Loss và Metrics (Top-K accuracy)
        # items_ds.batch(128).map(item_model) nghĩa là:
        # Khi tính toán, nó sẽ lấy toàn bộ video trong kho, chạy qua ItemTower 
        # để tạo ra index các vector ứng viên.
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=items_ds.map(item_model) 
            )
        )

    def compute_loss(self, features, training=False):
        # 1. Tính User Vector (Query)
        user_embeddings = self.user_model({
            "user_id": features["user_id"],
            "age": features["age"],
            "fre_city": features["fre_city"]
        })

        # 2. Tính Item Vector (Candidate) - Của chính video đang xem (Positive)
        positive_movie_embeddings = self.item_model({
            "pid": features["pid"],
            "item_vector": features["item_vector"]
        })

        # 3. Tính Loss (Contrastive Loss / Softmax Loss)
        # TFRS sẽ tự lấy các items khác trong batch làm Negative Samples (In-batch negative)
        return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
# Khởi tạo model tổng
model = YouTubeRetrievalModel(user_model, item_model)

# Compile (Dùng Adagrad hoặc Adam)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Train
# epoch nên để thấp (3-5) nếu dữ liệu lớn để tránh overfitting
model.fit(train_ds, epochs=5, validation_data=test_ds)

In [ ]:
# 1. Tạo công cụ tìm kiếm BruteForce (quét cạn - chính xác 100% nhưng chậm nếu > 1 triệu item)
# Nếu dữ liệu lớn, hãy dùng tfrs.layers.factorized_top_k.ScaNN (cần cài thêm thư viện scann)
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# 2. Đưa toàn bộ Item vào index (đã vector hóa)
index.index_from_dataset(
  tf.data.Dataset.zip((
      items_ds.map(lambda x: x["pid"]), # Chỉ lấy ID để trả về
      items_ds.map(model.item_model)    # Vector tương ứng
  ))
)

# 3. Test thử gợi ý
# Lấy 1 user mẫu từ tập test
sample_user = {
    "user_id": tf.constant(["12345"]), # Giả sử ID 12345
    "age": tf.constant([25.0]),
    "fre_city": tf.constant(["Hanoi"])
}

# Lấy 3 gợi ý tốt nhất
_, titles = index(sample_user, k=3)

print(f"Top 3 videos for user 12345: {titles[0].numpy()}")

In [2]:
import sys

# In toàn bộ sys.path để debug
print("Before filter, sys.path:", sys.path)

# Lọc: giữ lại phần tử là string đường dẫn hợp lệ (non-empty)
new_paths = []
for p in sys.path:
    if isinstance(p, str) and p:
        new_paths.append(p)
    else:
        print("Removing invalid path from sys.path:", p)

sys.path[:] = new_paths

print("After filter, sys.path:", sys.path)


Before filter, sys.path: ['C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311', 'd:\\learn\\giaotrinh\\ky7\\CV\\final\\cv_final_venv', '', 'd:\\learn\\giaotrinh\\ky7\\CV\\final\\cv_final_venv\\Lib\\site-packages']
Removing invalid path from sys.path: 
After filter, sys.path: ['C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\ADMIN\\AppData\\Local\\Programs\\Python\\Python311', 'd:\\learn\\giaotrinh\\ky7\\CV\\final\\cv_final_venv', 'd:\\learn\\giaotrinh\\ky7\\CV\\final\\cv_final_venv\\Lib\\site-packages']
